In [ ]:
import os, copy
import numpy as np

#-------------------

import torch.nn as nn
from torch.optim import Adam

#-------------------
import poseigen_seaside.basics as se
import poseigen_seaside.metrics as mex

import poseigen_compass as co
import poseigen_trident.utils as tu
import poseigen_oyster as oys

In [ ]:
data_path = "../data/R3/"
os.makedirs(data_path, exist_ok=True)

os.chdir(data_path)

In [ ]:
RO_folder = se.NewFolder('RO')

In [ ]:
fin_bins = se.PickleLoad('fin_bins')
divset = ['Train', 'Stop.', ' Eval', 'Test']

10


In [ ]:
yy1_split_transaug = se.PickleLoad('yy1_split_transaug')

yy1_max = se.PickleLoad('yy1_max')

tri_tpacks = se.PickleLoad('tri_tpacks')

d_x1, d_y_ms, d_s, d_b, d_x2 = tri_tpacks

d_s_rs = np.swapaxes(np.expand_dims(d_s, axis = -1), 1, -1)

In [ ]:
masterpseudo = 1e-10

RMS_mode = [mex.MeanExpo, {'expo': 2, 'root': True}]
RMS_mode_pyt = [mex.MeanExpo, {'expo': 2, 'root': True, 'pyt': True}]

bm_args_np = {'byaxis': 1, 'useweights': False, 'seperate': False, 
              'summarize_mode': RMS_mode}

bm_args_pyt = {'byaxis': 1, 'useweights': False, 'seperate': False, 
               'summarize_mode': RMS_mode_pyt, 'pyt': True}

#------------------------------------------------

deverr_args_base = {'expo': 1, 'root': False,
                    'pseudo': masterpseudo,
                    'scalefactor': yy1_max}

RMSDE_mode_np = [mex.DeviaError, {**deverr_args_base, 'pyt': False}]
RMSDE_mode_pyt = [mex.DeviaError, {**deverr_args_base, 'pyt': True}]  

B_RMSDE_mode_np = [tu.BinnedLoss, {'metrics_mode': RMSDE_mode_np, **bm_args_np}]
B_RMSDE_mode_pyt = [tu.BinnedLoss, {'metrics_mode': RMSDE_mode_pyt, **bm_args_pyt}]

In [6]:
d_y_ms_mean = np.mean(d_y_ms)
d_y_ms_mean_xpa = np.repeat(d_y_ms_mean, d_y_ms.shape[0]).reshape(d_y_ms.shape)
print(d_y_ms_mean)

#--------------------------------------------
d_y_ms_bind = [d_y_ms[d_b == i] for i in np.arange(fin_bins)]
d_y_ms_bind_mean = [np.mean(d, keepdims= True) for d in d_y_ms_bind]
d_y_ms_mom = np.mean(d_y_ms_bind_mean)
d_y_ms_mom_xpa = np.repeat(d_y_ms_mom, d_y_ms.shape[0]).reshape(d_y_ms.shape)

print(d_y_ms_mom)

#============================================

for x in [d_y_ms_mean_xpa, d_y_ms_mom_xpa]: 
    mex = B_RMSDE_mode_np[0](x, d_y_ms, std = d_s_rs, bind = d_b, 
                    ** B_RMSDE_mode_np[1])

    print(mex)

#+++++++++++++++++++++++++++++++++++++++++++++

d_y_ms_mean_sp = [np.mean(d_y_ms[s]) for s in yy1_split_transaug]
d_y_ms_mean_sp_xpa = [np.repeat(d_y_ms_mean_sp[iz], d_y_ms[z].shape[0]).reshape(d_y_ms[z].shape) 
                        for iz, z in enumerate(yy1_split_transaug)]
print(d_y_ms_mean_sp)


d_y_ms_sp_bind = [[d_y_ms[s][d_b[s] == i] for i in np.arange(fin_bins)] for s in yy1_split_transaug]
d_y_ms_sp_bind_mean = [[np.mean(d[i], keepdims= True) 
                        for i in np.arange(fin_bins)] 
                        for d in d_y_ms_sp_bind]

print(d_y_ms_sp_bind_mean) 
d_y_ms_sp_mom = [np.mean(d_y_ms_sp_bind_mean[iz]) for iz, z in enumerate(yy1_split_transaug)]
print(d_y_ms_sp_mom) 
d_y_ms_sp_mom_xpa = [np.repeat(d, d_y_ms[z].shape[0]).reshape(d_y_ms[z].shape) 
                    for z,d in zip(yy1_split_transaug, d_y_ms_sp_mom)]



for x in [d_y_ms_mean_sp_xpa, d_y_ms_sp_mom_xpa]:
    for iz, z in enumerate(yy1_split_transaug):

        mex = B_RMSDE_mode_np[0](x[iz], d_y_ms[z], std = d_s_rs[z], bind = d_b[z], 
                        ** B_RMSDE_mode_np[1])

        print(mex)

# print(d_y_ms_mom)

0.015893282
0.25834477
22.308273110898273
10.265206110288915
[np.float32(0.016669521), np.float32(0.018074794), np.float32(0.011771729), np.float32(0.014987123)]
[[array([0.00721407], dtype=float32), array([0.05568025], dtype=float32), array([0.09851599], dtype=float32), array([0.14139737], dtype=float32), array([0.19063948], dtype=float32), array([0.2440531], dtype=float32), array([0.30516747], dtype=float32), array([0.38254878], dtype=float32), array([0.4881846], dtype=float32), array([0.67247105], dtype=float32)], [array([0.00721983], dtype=float32), array([0.05559878], dtype=float32), array([0.0996116], dtype=float32), array([0.14216818], dtype=float32), array([0.19122344], dtype=float32), array([0.23924538], dtype=float32), array([0.29826853], dtype=float32), array([0.385225], dtype=float32), array([0.4925386], dtype=float32), array([0.68055236], dtype=float32)], [array([0.0056862], dtype=float32), array([0.05475389], dtype=float32), array([0.10015788], dtype=float32), array([0.14

In [ ]:
flipo = [[-2, -1], [-2, -1]]

TT_args =  {'loss_mode': B_RMSDE_mode_pyt, 'loss_bind': True, 
            'metrics_mode': B_RMSDE_mode_np, 'smallest': True,

            'batchsize': 128, 'opt': Adam, 'learningrate': 0.002, 'maxepochs': 100, 
            'patience': 8, 'pickup': True,

            'flips': flipo, #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            'indivflips': True}


#-------------------------------------------------

# RANDOM OPT ARGS: #

TT_args_RO = TT_args.copy() 
TT_args_RO.update({'inps': [d_x1, d_x2], 'out': d_y_ms, 
                   'out_std': d_s_rs, 'out_bind': d_b,
                   'Split': yy1_split_transaug,                                   ##########
                   'EUS': 0.1, 'obs_weight': d_b.reshape(-1)})               ##########

#----------------------------------------

TCS_args_RO = {'trainer': tu.TridentTrainer, 
               #'trainer_args': TT_args_RO, # NEED TO DO THIS FOR EACH MODEL FOR DUDS.  
               'smallest': None,
               'get_predictions': True, 'pred_rewrite': False, 
               'add_pred_args': {'batchsize': 512, 'flips': flipo, 'avg_flips': True},
               'score_on': 2, 'score_only': True} ### SCORING ON THE EVAL SET, MAKING ONMLY PREDS FOR IT. 


########################

base_dict = {'batchsize': [[128, 256, 512], 'cat'], 
                'learningrate': [se.GeomNumSpacing(0.0001, 0.01, 15, plier = 1.2), 'cat']} ####################

######################

budget = 200                                        #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

RO_repeats = 1

RO_args_shared = {
           'data': None, ######################
           'Splits': None, ####################
           'budget': budget, 'repeats': RO_repeats, 
           #'CS_mode': [tu.TridentCanScorer, TCS_args_RO], # NEED TO SET THIS INDIV FOR DUDS. 
           'pickup': True, 'savemodels': True}

#---------------------------------------------------------------------------

TC_repeats = 10

TCR_args_RO = {'Splits': None, 'repeats': TC_repeats,
               'pickup': False, 'savemodels': True, 'returnmodel': True}        

In [8]:
def Reset_DualOyster(dualoyster):
    #ds consists of a "conv" and a "dense" module. Need to go through each one, see if its a conv and reset if so. 

    for oyster in [dualoyster.OysterA, dualoyster.OysterB]: 
        lke, los = len(oyster.kE), len(oyster.O)

        for i in np.arange(lke): 
            if isinstance(oyster.kE[i], nn.Conv2d): 
                oyster.kE[i].reset_parameters()
        
        for i in np.arange(los): 
            if isinstance(oyster.O[i], nn.Conv2d): 
                oyster.O[i].reset_parameters()
        print('done reset mod')

    return dualoyster

def GetTopofTop(pn_RO, algo, 
                duds_mode = [Reset_DualOyster, {}], 
                rewrite = False): 

    topcan = co.TopCan(pn_RO, num_top = 5, 
                       reduce_func = np.min, smallest = True, perround = False)
 
    topcan_cd = co.TopCanDict(pn_RO, topcan, comb_rounds = True)

    pn_RO_top = se.NewFolder(pn_RO + 'Top' + str(0))

    #----------------------------------------------------

    TT_args_TC = {'duds_mode': duds_mode, 'duds': 10, **TT_args_RO}

    TC_repeats = 10
    
    TCR_args_RO = {'Splits': None, 'repeats': TC_repeats,
                   'pickup': False, 'savemodels': True, 'returnmodel': True}


    TCS_args_TC = {'trainer': tu.TridentTrainer, 'smallest': None,
                   'trainer_args': TT_args_TC,
                   
                   'get_predictions': True, 'pred_rewrite': False, 
                   'add_pred_args': {'batchsize': 512, 'flips': flipo, 'avg_flips': True},
                   'score_on': 1, 'score_only': False}    

    for no, ca in topcan_cd.items():
    
        pn_t = se.NewFolder(pn_RO_top + no)
        
        TCR_args_rep = {**TCR_args_RO, **TCS_args_TC, 'pathname': pn_t}
        
        _, _= tu.TridentCanRepeater(algo, ca, data = None, **TCR_args_rep)

    
    #=================================================================

    es_args = {'out': d_y_ms, 'out_std': d_s_rs, 'out_bind': d_b,
                'split': yy1_split_transaug,
                'metrics_mode': B_RMSDE_mode_np, 'score_on': 1,
                'std_cutoff': None, 'ddof': 1, 'top': 3, 'smallest': True}  

    boots_scores = []

    for no, ca in topcan_cd.items():
        
        pn_t = se.NewFolder(pn_RO_top + no)

        pn_preds = [pn_t + '0_' + str(ir) + '_Preds' + '.p' ###################
                    for ir in range(TC_repeats)]
        
        pn_e = pn_t + 'boots_ensemb'

        if os.path.isfile(pn_e + '.p') and rewrite is False: 
            boots_ensemb = se.PickleLoad(pn_e)
        else: 
            boots_ensemb = co.Bootstrapper(pn_preds, mode = [co.EnsembleScorer, es_args],
                                            iters = 100, updates = 10, return_idx = True)
            se.PickleDump(boots_ensemb, pn_e)  

        boots_scores.append(boots_ensemb[0])
    
    boots_scores = np.array(boots_scores)


    boots_means = np.mean(boots_scores, axis = 1)
    boots_means_eval = boots_means[:, 2]
    print(boots_means_eval)
    sortxo = np.argsort(boots_means_eval)
    sortxo_keys = np.array(list(topcan_cd.keys()))[sortxo]
    ranked_can_dict = {ix: topcan_cd[x] for ix, x in enumerate(sortxo_keys)}

    #se.PickleDump(ranked_can_dict, pn_RO_top + 'ranked_can_dict')

    return ranked_can_dict

In [ ]:
DualOyster_BaseDict = {'A_dim_i': [[(1,249,4)],'cat'], 
                      'B_dim_i': [[(11,40,1)],'cat'],
                      'dim_f': [[(2,1,1)],'cat'], 

                      'A_kE_ck_grouped': [[True, False], 'cat'], #NEED TO BE SET
                      'A_DS': [[True, False], 'cat'],
                      'A_kE_k': [[12, 20], 'int'], 
                      'A_kE_cf_m': [[50, 150], 'int'],
                      'A_kE_cf_ns': [[1, 1.1, 1.2], 'cat'],
                      'A_kE_ck_base': [[4, 5, 6, 7, 8, None], 'cat'], 
                      'A_kE_OneByOne': [[True, False], 'cat'],
                      'A_P_ck': [[6, 20], 'int'],
                      
                      'B_kE_ck_grouped': [[True, False], 'cat'], #NEED TO BE SET
                      'B_DS': [[True, False], 'cat'],
                      'B_kE_k': [[12, 20], 'int'], 
                      'B_kE_cf_m': [[50, 150], 'int'],
                      'B_kE_cf_ns': [[1, 1.1, 1.2], 'float'],
                      'B_kE_ck_base': [[4, 5, 6, 7, 8, None], 'cat'], 
                      'B_kE_OneByOne': [[True, False], 'cat'],
                      'B_P_ck': [[6, 20], 'int'],

                      'DS_wwp_func': [[nn.MaxPool2d], 'cat'],
                      'DS_act': [[False], 'cat'],

                      'P_center': [[True, False], 'cat'],

                      'O_mods': [[None], 'cat'],
                      'O_mods_ns': [[0.5], 'cat'],
                      'O_cf_pu_m': [[None], 'cat'],
                      'O_cf_ns': [[1, 1.1, 1.2], 'cat'],
                      'O_ck_base': [[None], 'cat'],
                      'O_pool_k2s': [[0, 0.5], 'cat'],
                      'O_pool_func': [[nn.MaxPool2d], 'cat'],
                      'O_actb4pool': [[True], 'cat'],
                      'O_dropout': [[0, 0.2], 'cat'],


                      'activations': [[nn.ReLU(), nn.LeakyReLU()], 'cat'],
                      'batchnorm': [[None, 'before'], 'cat']}

#============================================================

optdict = {**DualOyster_BaseDict, **base_dict}

optdict.update({
    'A_dim_i': [[d_x1.shape[1:]],'cat'],
    'B_dim_i': [[d_x2.shape[1:]],'cat'],
    'dim_f': [[d_y_ms.shape[1:]],'cat'],

    'A_kE_ck_grouped': [[False], 'cat'], 

    'A_DS': [[False], 'cat'],                               #!!!!!!!!!!!!!!     
    'A_kE_k': [[11, 13, 15], 'cat'], 
    'A_kE_cf_m': [[50, 150, 250], 'cat'],
    'A_kE_cf_ns': [[1, 1.15, 1.3], 'cat'],
    'A_kE_ck_base': [[None, 7, 9], 'cat'],
    'A_kE_OneByOne': [[True, False], 'cat'],
    'A_P_ck': [[1, 25, 50, 100], 'cat'],

    'B_kE_ck_grouped': [[True], 'cat'], #DOESNT MATTER
    
    'B_DS': [[False], 'cat'],
    'B_kE_k': [[5, 10, 15], 'cat'], 
    'B_kE_cf_m': [[10, 20, 30], 'cat'],
    'B_kE_cf_ns': [[1, 1.15, 1.3], 'cat'],
    'B_kE_ck_base': [[None, 5, 7, 9], 'cat'],
    'B_kE_OneByOne': [[True, False], 'cat'],
    'B_P_ck': [[1, 5, 10], 'cat'],                   #!!!!!!!!!!!!!!
    
    'P_center': [[False, True], 'cat'],

    'O_dropout': [[0, 0.2], 'cat'],
    
    'O_mods': [[0], 'cat'],

    'O_dropout': [[0, 0.2], 'cat'],

    'O_mods_ns': [[0.5], 'cat'],
    'O_cf_pu_m': [[None], 'cat'],
    'O_cf_ns': [[1], 'cat'],
    'O_ck_base': [[None], 'cat'],
    'O_pool_k2s': [[0], 'cat'],
    'O_pool_func': [[nn.MaxPool2d], 'cat'],
    'O_actb4pool': [[True], 'cat']})

In [10]:
modo = oys.DualOyster

TT_args_RO_oys = {'duds_mode': [Reset_DualOyster, {}], 'duds': 10,
                 **TT_args_RO}
TCS_args_RO_oys = {'trainer_args': TT_args_RO_oys,
                  **TCS_args_RO}

#--------------------------------------

RO_args_oys = {'algo': modo,
              'CS_mode': [tu.TridentCanScorer, TCS_args_RO_oys],
                **RO_args_shared}

# Exact

In [ ]:
optdict_exact = copy.deepcopy(optdict)
optdict_exact.update({'O_mods': [[0], 'cat']})

In [ ]:
ex_oys_folder = se.NewFolder(RO_folder + 'exact')

oys_iter = 11

pn_RO_oys = se.NewFolder(ex_oys_folder + 'RO_' + str(oys_iter))

In [13]:
ROO_oys = co.RandomOpt(**RO_args_oys, 
                       VarDict = optdict_exact, pathname = pn_RO_oys)

{0: {0: {0: np.float64(6.053819566595472)}}, 1: {0: {0: np.float64(5.841409100867844)}}, 2: {0: {0: np.float64(5.591365391840631)}}, 3: {0: {0: np.float64(3.714976486077722)}}, 4: {0: {0: np.float64(3.5159822642167544)}}, 5: {0: {0: np.float64(5.604719381964225)}}, 6: {0: {0: np.float64(5.889943214518365)}}, 7: {0: {0: np.float64(6.116185528746612)}}, 8: {0: {0: np.float64(84.47103327610326)}}, 9: {0: {0: np.float64(6.020622419675992)}}, 10: {0: {0: np.float64(6.096102122459192)}}, 11: {0: {0: np.float64(5.442394751852653)}}, 12: {0: {0: np.float64(13.240332690950115)}}, 13: {0: {0: np.float64(5.711572400376335)}}, 14: {0: {0: np.float64(5.500905108741471)}}, 15: {0: {0: np.float64(6.784154292423523)}}, 16: {0: {0: np.float64(5.709453826517666)}}, 17: {0: {0: np.float64(5.506595915088782)}}, 18: {0: {0: np.float64(5.7672267850795045)}}, 19: {0: {0: np.float64(135.45722653084687)}}, 20: {0: {0: np.float64(3.7786378199533446)}}, 21: {0: {0: np.float64(7.758395419731611)}}, 22: {0: {0: np

In [ ]:
rankedo = GetTopofTop(pn_RO_oys, modo, 
                      duds_mode = [Reset_DualOyster, {}], 
                      rewrite = False)

se.PickleDump(rankedo, pn_RO_oys + 'ranked_can_dict')

In [16]:
rankedo[0]

{'A_dim_i': (1, 500, 4),
 'B_dim_i': (11, 40, 1),
 'dim_f': (1, 1, 1),
 'A_kE_ck_grouped': False,
 'A_DS': False,
 'A_kE_k': 15,
 'A_kE_cf_m': 150,
 'A_kE_cf_ns': 1.15,
 'A_kE_ck_base': 7,
 'A_kE_OneByOne': True,
 'A_P_ck': 50,
 'B_kE_ck_grouped': True,
 'B_DS': False,
 'B_kE_k': 10,
 'B_kE_cf_m': 30,
 'B_kE_cf_ns': 1,
 'B_kE_ck_base': 9,
 'B_kE_OneByOne': True,
 'B_P_ck': 5,
 'DS_wwp_func': torch.nn.modules.pooling.MaxPool2d,
 'DS_act': False,
 'P_center': True,
 'O_mods': 0,
 'O_mods_ns': 0.5,
 'O_cf_pu_m': None,
 'O_cf_ns': 1,
 'O_ck_base': None,
 'O_pool_k2s': 0,
 'O_pool_func': torch.nn.modules.pooling.MaxPool2d,
 'O_actb4pool': True,
 'O_dropout': 0.2,
 'activations': ReLU(),
 'batchnorm': 'before',
 'batchsize': 256,
 'learningrate': np.float64(0.00671949253976686)}

# Non Exact

In [17]:
optdict_ne = copy.deepcopy(optdict)

optdict_ne.update({'O_mods': [[2, 3], 'cat'],
                    'O_mods_ns': [[0, 0.3, 0.5], 'cat'],
                    'O_cf_pu_m': [[1], 'cat'],
                    'O_cf_ns': [[1], 'cat'],
                    'O_ck_base': [[3, 5], 'cat'],
                    'O_pool_k2s': [[0.5], 'cat'],
                    'O_pool_func': [[nn.MaxPool2d], 'cat'],
                    'O_actb4pool': [[True], 'cat']})

In [ ]:
nonex_oys_folder = se.NewFolder(RO_folder + 'nonexact')

oys_iter = 11

pn_RO_oys = se.NewFolder(nonex_oys_folder + 'RO_' + str(oys_iter))

In [ ]:
ROO_ne = co.RandomOpt(**RO_args_oys, 
                       VarDict = optdict_ne, pathname = pn_RO_oys)

{0: {0: {0: np.float64(4.863062218899112)}}, 1: {0: {0: np.float64(5.998567034163008)}}, 2: {0: {0: np.float64(6.3781893729484)}}, 3: {0: {0: np.float64(6.2101108929384985)}}, 4: {0: {0: np.float64(6.129360331934509)}}, 5: {0: {0: np.float64(5.610576412630549)}}, 6: {0: {0: np.float64(6.173528134516517)}}, 7: {0: {0: np.float64(6.855741920480608)}}, 8: {0: {0: np.float64(5.890491416294675)}}, 9: {0: {0: np.float64(3.8169115510278986)}}, 10: {0: {0: np.float64(5.407448280765756)}}, 11: {0: {0: np.float64(5.574333333576667)}}, 12: {0: {0: np.float64(5.675741829222865)}}, 13: {0: {0: np.float64(5.698507592372613)}}, 14: {0: {0: np.float64(3.685435487498009)}}, 15: {0: {0: np.float64(5.439350326329895)}}, 16: {0: {0: np.float64(6.202571306379211)}}, 17: {0: {0: np.float64(5.519571768623336)}}, 18: {0: {0: np.float64(6.654821238104094)}}, 19: {0: {0: np.float64(7.169865256616204)}}, 20: {0: {0: np.float64(6.618762099316944)}}, 21: {0: {0: np.float64(5.436990542717231)}}, 22: {0: {0: np.floa

In [ ]:
rankedo = GetTopofTop(pn_RO_oys, modo, rewrite = False)

se.PickleDump(rankedo, pn_RO_oys + 'ranked_can_dict')

In [21]:
rankedo

{0: {'A_dim_i': (1, 500, 4),
  'B_dim_i': (11, 40, 1),
  'dim_f': (1, 1, 1),
  'A_kE_ck_grouped': False,
  'A_DS': False,
  'A_kE_k': 13,
  'A_kE_cf_m': 150,
  'A_kE_cf_ns': 1.3,
  'A_kE_ck_base': 9,
  'A_kE_OneByOne': True,
  'A_P_ck': 25,
  'B_kE_ck_grouped': True,
  'B_DS': False,
  'B_kE_k': 15,
  'B_kE_cf_m': 20,
  'B_kE_cf_ns': 1,
  'B_kE_ck_base': 7,
  'B_kE_OneByOne': True,
  'B_P_ck': 10,
  'DS_wwp_func': torch.nn.modules.pooling.MaxPool2d,
  'DS_act': False,
  'P_center': False,
  'O_mods': 2,
  'O_mods_ns': 0,
  'O_cf_pu_m': 1,
  'O_cf_ns': 1,
  'O_ck_base': 3,
  'O_pool_k2s': 0.5,
  'O_pool_func': torch.nn.modules.pooling.MaxPool2d,
  'O_actb4pool': True,
  'O_dropout': 0,
  'activations': LeakyReLU(negative_slope=0.01),
  'batchnorm': None,
  'batchsize': 256,
  'learningrate': np.float64(0.008210632294418286)},
 1: {'A_dim_i': (1, 500, 4),
  'B_dim_i': (11, 40, 1),
  'dim_f': (1, 1, 1),
  'A_kE_ck_grouped': False,
  'A_DS': False,
  'A_kE_k': 13,
  'A_kE_cf_m': 250,
  'A_